In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [5]:
def getAdjustment():
    method = 'hmy_getCurrentUtilityMetrics'
    params = []
    num = get_information(method, params)['result']['Adjustment']
    return float(num)

In [6]:
def getBlockSigners(blockNum):
    method = 'hmy_getBlockSigners'
    params = [blockNum]
    return get_information(method, params)['result']

In [7]:
def R6_test():
    print("Test-R6: Tests whether the delegation reward is distributed correctly")
    num = int(input("enter the number of blocks you want to test: "))
    
    current_block = getBlockNumber()
    next_block = current_block + 1
    
    acc_rewards_prev = dict()
    delegations_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            ds = i['validator']['delegations']
            dels = dict()
            for d in ds:
                d_addr = d['delegator-address']
                d_reward = d['reward']
                dels[d_addr] = d_reward
            delegations_prev[address] = dels
            
    current_block = getBlockNumber()
    while current_block < next_block:
        current_block = getBlockNumber()
    
    iterations = 0
    while iterations < num:
        print("current block", current_block)
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        delegations_curr = dict()
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                acc_rewards_curr[address] = reward_accumulated
                if address not in acc_rewards_prev:
                    continue
                reward = reward_accumulated - acc_rewards_prev[address]
                if reward == 0:
                    continue
                elif reward < 0:
                    reward = -reward # first time delegations
                commission = float(i['validator']['rate']) * reward
                total_delegation_reward = reward - commission
                total_delegation = i['total-delegation']
                ds = i['validator']['delegations']
                del_rewards = 0
                dels = delegations_prev[address]
                dels_curr = dict()
                for d in ds:
                    d_addr = d['delegator-address']
                    d_reward = d['reward']
                    dels_curr[d_addr] = d_reward
                    d_amount = d['amount']
                    delegation_reward_actual = d_reward
                    if d_addr in dels:
                        delegation_reward_actual = delegation_reward_actual - dels[d_addr]
                    percentage = d_amount / total_delegation
                    delegation_reward_expected = int(percentage * total_delegation_reward)
                    if d_addr == address:
                        delegation_reward_expected = delegation_reward_expected + commission
                    if "{:.15e}".delegation_reward_actual != "{:.15e}".delegation_reward_expected:
                        print("Test R6: Fail for validator", address, "delegation", d_addr, \
                            "expected", "{:.15e}".delegation_reward_expected, \
                            "actual", "{:.15e}".format(delegation_reward_actual))
                delegations_curr[address] = dels_curr
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        acc_rewards_prev = acc_rewards_curr
        delegations_prev = delegations_curr
              
        iterations = iterations + 1

In [8]:
R6_test()

Test-R6: Tests whether the delegation reward is distributed correctly
enter the number of blocks you want to test: 1
current block 815
Test R6: Fail for validator one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 delegation one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 expected 9.787693966485529e+18 actual 9.78769396648552857600e+18
Test R6: Fail for validator one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 delegation one1km7xg8e3xjys7azp9f4xp8hkw79vm2h3f2lade expected 257217407943381216 actual 2.57217407943381184000e+17
Test R6: Fail for validator one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 delegation one1gm8xwupd9e77ja46eaxv8tk4ea7re5zknaauvq expected 771652223830143616 actual 7.71652223830143616000e+17
